In [9]:
import pandas as pd
import numpy as np

# Import des datasets

## Dataset universitaire de base (toutes les années universitaires)

In [10]:
df_uni_base = pd.read_csv('../nba_forecast/data/df_csv_no_url.csv')

## Dataset des performences athletics

In [11]:
df_perf_athletics = pd.read_csv('../nba_forecast/data/perf_athletics.csv')

## Dataset universitaire (dernière année)

In [12]:
df_uni_last_year = pd.read_csv('../nba_forecast/data/last_ncaa_season_bb.csv')

## Dataset des joueurs universitaires sans performences athlétiques avec leur position

In [13]:
string_uni_players_without_athletics  = "A.J. Hammons: C, Aaron White: PF, Abdel Nader: SG, Alex Oriakhi: PF,\
Andrew Wiggins: SF, Anthony Bennett: PF,\
Arsalan Kazemi: SF, Bam Adebayo: C, Ben Simmons: PF, Bernard James: C, Brandon Ingram: SF, C.J. McCollum: SG, Cady Lalanne: C,\
Chandler Hutchinson: SF, Darrun Hilliard: SG, De'Anthony Melton: PG, Deandre Ayton: C, Dejounte Murray: PG, Dennis Smith Jr: PG,\
Devonte' Graham: PG, Domantas Sabonis: C, Frank Mason: PG, Gary Trent Jr.: PG, Glen Rice Jr: SG, Jabari Bird: SG,\
Jabari Parker: PF, Jahlil Okafor: C, Jamal Murray: PG, Jaren Jackson Jr. : PF, Jarred Vanderbilt: PF, Jayson Tatum: SF,\
Jeff Taylor: SF, Joe Harris: SF, Joe Young: PG, Joel Embiid: C, Jonah Bolden: C, Jonathan Isaac: PF, Josh Huestis: SG,\
Josh Jackson: SF, Josh Richardson: SG, Justin Hamilton: C, Karl-Anthony Towns: C, Lauri Markkanen: PF, Lonzo Ball: PG,\
Malik Monk: SG, Marvin Bagley III : PF, Michael Porter Jr. : SF, Mikal Bridges: SF, Mitch McGary: PF, Ray Spalding: PF,\
Robert Sacre: C, Robert Williams: C, Romero Osby: SF, Roy Devyn Marble: Missing, Sir 'Dominic Pointer: SF,\
Stephen Zimmerman Jr. : C, Sterling Brown: SG, Tyrone Wallace : PG, Vince Edwards: SF, Wendell Carter Jr. : C"

In [14]:
list_string_uni_players_without_athletics = [list(map(lambda word:word.strip(), x.split(':'))) \
                                               for x in string_uni_players_without_athletics.split(',')]

In [15]:
list_player_name = np.array(list_string_uni_players_without_athletics)[:,:1].reshape(-1)

In [16]:
list_position = np.array(list_string_uni_players_without_athletics)[:,1:].ravel()

In [17]:
dico_uni_players_without_athletics = {'player_name':list_player_name,'position':list_position}

In [18]:
df_uni_players_without_athletics  = pd.DataFrame(dico_uni_players_without_athletics)

In [19]:
df_uni_players_without_athletics.head()

,player_name,position
0,A.J. Hammons,C
1,Aaron White,PF
2,Abdel Nader,SG
3,Alex Oriakhi,PF
4,Andrew Wiggins,SF


# Vérification sur les Datasets


In [20]:
df_uni_base['player_name'].nunique()

334

In [21]:
df_uni_last_year['player_name'].nunique()

334

# Correspondance des noms des joueurs entre le Dataset universitaire ( dernière année) et le Dataset performences athlétics

## Noms à corriger dans le Dataset *df_perf_athletics*

In [22]:
df_perf_athletics[df_perf_athletics['PLAYER_NAME'].str.contains('[^a-zA-Z0-9^\s]', regex=True)]['PLAYER_NAME']

4                 J'Covan Brown
23          Darius Johnson-Odom
28       Michael Kidd-Gilchrist
43                 Kyle O'Quinn
66     Kentavious Caldwell-Pope
69      Michael Carter-Williams
83             Tim Hardaway Jr.
143                   C.J. Fair
146               P.J. Hairston
157              K.J. McDaniels
161         Johnny O?Bryant III
175                 T.J. Warren
176                 C.J. Wilcox
186         Willie Cauley-Stein
201     Rondae Hollis-Jefferson
204                 R.J. Hunter
225            D?Angelo Russell
228                 J.P. Tokoto
257         Dorian Finney-Smith
302                V.J. Beachem
318                De?Aaron Fox
326        Justin Jackson (UMD)
327        Justin Jackson (UNC)
352          Derrick Walton Jr.
355         Nigel Williams-Goss
356                 D.J. Wilson
364            Keita Bates-Diop
382     Shai Gilgeous-Alexander
Name: PLAYER_NAME, dtype: object

In [23]:
df_perf_athletics[df_perf_athletics['PLAYER_NAME'].str.contains('Devonte', regex=True)]['PLAYER_NAME']

383    Devonte Graham
Name: PLAYER_NAME, dtype: object

In [24]:
df_perf_athletics.loc[161,'PLAYER_NAME'] = "Johnny O'Bryant"

In [25]:
df_perf_athletics.loc[225,'PLAYER_NAME'] = "D'Angelo Russell"

In [26]:
df_perf_athletics.loc[318,'PLAYER_NAME'] = "De'Aaron Fox"

## Noms à corriger dans le Dataset *df_uni_last_year*

In [27]:
df_uni_last_year[df_uni_last_year['player_name'].str.contains('Justin Jackson', regex=False)]['player_name']

259    Justin Jackson
335    Justin Jackson
Name: player_name, dtype: object

In [28]:
df_uni_last_year.iloc[[259,335]]

,player_name,season,school_name,conf_abbr,g,gs,mp,per,ts_pct,efg_pct,...,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,years
259,Justin Jackson,2016-17,UNC,ACC,40,39,1281,21.3,0.555,0.531,...,9.5,25.7,4.3,1.8,6.1,0.191,7.7,2.1,9.8,3
335,Justin Jackson,2017-18,Maryland,Big Ten,11,10,321,14.2,0.470,0.416,...,21.8,24.1,0.0,0.6,0.7,0.083,0.2,1.7,1.9,2


In [29]:
df_uni_last_year.loc[259, 'player_name'] = 'Justin Jackson (UNC)'

In [30]:
df_uni_last_year.loc[335, 'player_name'] = 'Justin Jackson (UMD)'

# Concaténation de *df_perf_athletics* & *df_uni_players_without_athletics* et traitement des doublons

Le nom des joueurs (colonne 'player_name') est la clé utilisée pour merger l'ensemble des Datasets

## Traitement préalable sur les Datasets (drop columns inutiles)

In [31]:
df_perf_athletics.drop(columns='Unnamed: 0', inplace=True)

In [32]:
df_perf_athletics.rename(columns={'PLAYER_NAME': 'player_name', 'POSITION': 'position'}, inplace=True)

## Concaténation de *df_perf_athletics* et *df_uni_players_without_athletics* 

In [33]:
df_athletics = pd.concat([df_perf_athletics,df_uni_players_without_athletics], axis=0, ignore_index=True)

In [34]:
df_athletics

,year,player_name,position,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,HEIGHT_WO_SHOES,HEIGHT_W_SHOES,STANDING_REACH,WEIGHT,WINGSPAN
0,2012-13,Quincy Acy,PF,7.10%,9.50,11.00,6' 6.5'',6' 7.75'',8' 10.5'',223.8,7' 2.75''
1,2012-13,Harrison Barnes,SF-SG,9.60%,9.00,8.50,6' 7'',6' 8'',8' 5.5'',227.8,6' 11.25''
2,2012-13,Will Barton,SG-SF,4.10%,9.00,9.50,6' 5'',6' 6'',8' 6.5'',174.4,6' 9.75''
3,2012-13,Bradley Beal,SG,6.00%,8.50,9.00,6' 3.25'',6' 4.75'',8' 4'',201.8,6' 8''
4,2012-13,J'Covan Brown,PG,12.50%,7.75,8.25,6' 1'',6' 2.25'',8' 0'',202.4,6' 5.25''
...,...,...,...,...,...,...,...,...,...,...,...
482,NaN,Stephen Zimmerman Jr.,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
483,NaN,Sterling Brown,SG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484,NaN,Tyrone Wallace,PG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
485,NaN,Vince Edwards,SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df_athletics['player_name'].nunique()

472

## Identification des doublons, traitement au cas par cas

In [36]:
df_athletics[df_athletics['player_name'].duplicated()]

,year,player_name,position,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,HEIGHT_WO_SHOES,HEIGHT_W_SHOES,STANDING_REACH,WEIGHT,WINGSPAN
100,2013-14,Tony Mitchell,PF-SF,5.55%,9.00,10.00,6' 7.5'',6' 8.75'',8' 10.5'',236.4,7' 2.5''
304,2017-18,Jaron Blossomgame,SF,5.70%,8.75,10.25,6' 5.75'',6' 6.75'',8' 8.5'',218.6,6' 10''
321,2017-18,Josh Hart,SG-SF,5.80%,8.50,9.50,6' 3.75'',6' 5'',8' 5'',209.2,6' 8.25''
322,2017-18,Nigel Hayes,PF,11.50%,8.75,10.00,6' 6.25'',6' 7.5'',8' 8.5'',254.2,7' 3.25''
348,2017-18,Caleb Swanigan,PF,-%,9.50,10.25,6' 7.5'',6' 8.5'',9' 0'',245.6,7' 3''
350,2017-18,Melo Trimble,PG,10.60%,8.00,8.75,6' 1.25'',6' 2.5'',7' 11.5'',195.2,6' 2''
358,2018-19,Rawle Alkins,SG,8.90%,8.50,10.00,6' 2.75'',6' 4.25'',8' 3'',217.4,6' 8.75''
375,2018-19,Hamidou Diallo,SG,4.45%,8.75,9.00,6' 4.25'',6' 6'',8' 7'',197.0,6' 11.5''
391,2018-19,Justin Jackson,PF,8.90%,9.00,9.50,6' 6'',6' 6.75'',8' 9'',229.4,7' 3''
404,2018-19,Malik Newman,SG,6.25%,8.50,9.50,6' 2.5'',6' 3.25'',8' 2.5'',189.2,6' 5.5''


In [37]:
df_athletics[df_athletics['player_name'] == 'Justin Jackson']

,year,player_name,position,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,HEIGHT_WO_SHOES,HEIGHT_W_SHOES,STANDING_REACH,WEIGHT,WINGSPAN
263,2016-17,Justin Jackson,SF,7.60%,8.75,8.75,6' 7'',6' 8.25'',8' 7.5'',193.4,6' 10.75''
391,2018-19,Justin Jackson,PF,8.90%,9.00,9.50,6' 6'',6' 6.75'',8' 9'',229.4,7' 3''


In [38]:
df_athletics[df_athletics['player_name'].str.contains('Justin Jackson', regex=False)]['player_name']

263          Justin Jackson
326    Justin Jackson (UMD)
327    Justin Jackson (UNC)
391          Justin Jackson
Name: player_name, dtype: object

In [39]:
df_athletics.iloc[[263,326,327,391]]

,year,player_name,position,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,HEIGHT_WO_SHOES,HEIGHT_W_SHOES,STANDING_REACH,WEIGHT,WINGSPAN
263,2016-17,Justin Jackson,SF,7.60%,8.75,8.75,6' 7'',6' 8.25'',8' 7.5'',193.4,6' 10.75''
326,2017-18,Justin Jackson (UMD),SF,7.10%,9.00,9.50,6' 5.75'',6' 7'',8' 11'',219.4,7' 3.25''
327,2017-18,Justin Jackson (UNC),SF,8.10%,8.75,9.25,6' 7'',6' 8.25'',8' 8.5'',200.8,6' 11''
391,2018-19,Justin Jackson,PF,8.90%,9.00,9.50,6' 6'',6' 6.75'',8' 9'',229.4,7' 3''


In [40]:
df_athletics.drop([263,391], inplace=True)

In [41]:
df_athletics[df_athletics['player_name'].str.contains('Justin Jackson', regex=False)]['player_name']

326    Justin Jackson (UMD)
327    Justin Jackson (UNC)
Name: player_name, dtype: object

## Drop des doublons identifiés

In [42]:
df_athletics[df_athletics['player_name'].duplicated()]['player_name'].values

array(['Tony Mitchell', 'Jaron Blossomgame', 'Josh Hart', 'Nigel Hayes',
       'Caleb Swanigan', 'Melo Trimble', 'Rawle Alkins', 'Hamidou Diallo',
       'Malik Newman', 'Moritz Wagner', 'Bernard James', 'Jeff Taylor',
       'Joe Harris', 'Ray Spalding'], dtype=object)

### Doublons identifiés

In [43]:
df_athletics[df_athletics['player_name'].duplicated()]

,year,player_name,position,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,HEIGHT_WO_SHOES,HEIGHT_W_SHOES,STANDING_REACH,WEIGHT,WINGSPAN
100,2013-14,Tony Mitchell,PF-SF,5.55%,9.00,10.00,6' 7.5'',6' 8.75'',8' 10.5'',236.4,7' 2.5''
304,2017-18,Jaron Blossomgame,SF,5.70%,8.75,10.25,6' 5.75'',6' 6.75'',8' 8.5'',218.6,6' 10''
321,2017-18,Josh Hart,SG-SF,5.80%,8.50,9.50,6' 3.75'',6' 5'',8' 5'',209.2,6' 8.25''
322,2017-18,Nigel Hayes,PF,11.50%,8.75,10.00,6' 6.25'',6' 7.5'',8' 8.5'',254.2,7' 3.25''
348,2017-18,Caleb Swanigan,PF,-%,9.50,10.25,6' 7.5'',6' 8.5'',9' 0'',245.6,7' 3''
350,2017-18,Melo Trimble,PG,10.60%,8.00,8.75,6' 1.25'',6' 2.5'',7' 11.5'',195.2,6' 2''
358,2018-19,Rawle Alkins,SG,8.90%,8.50,10.00,6' 2.75'',6' 4.25'',8' 3'',217.4,6' 8.75''
375,2018-19,Hamidou Diallo,SG,4.45%,8.75,9.00,6' 4.25'',6' 6'',8' 7'',197.0,6' 11.5''
404,2018-19,Malik Newman,SG,6.25%,8.50,9.50,6' 2.5'',6' 3.25'',8' 2.5'',189.2,6' 5.5''
420,2018-19,Moritz Wagner,C-PF,6.75%,9.00,10.75,6' 10.5'',6' 11.5'',9' 0'',241.4,7' 0''


### Drop des doublons

In [44]:
df_athletics[df_athletics['player_name'].str.contains('Ray Spalding', regex=False)]['player_name']

415    Ray Spalding
476    Ray Spalding
Name: player_name, dtype: object

In [45]:
df_athletics.loc[[415,476]]

,year,player_name,position,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,HEIGHT_WO_SHOES,HEIGHT_W_SHOES,STANDING_REACH,WEIGHT,WINGSPAN
415,2018-19,Ray Spalding,PF,5.55%,9.25,9.00,6' 9'',6' 10.25'',9' 1'',215.4,7' 4.75''
476,NaN,Ray Spalding,PF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
df_athletics.drop([244], inplace=True) #Jaron Blossomgame
df_athletics.drop([260], inplace=True) #Josh Hart
df_athletics.drop([261], inplace=True) #Nigel Hayes
df_athletics.drop([348], inplace=True) #Caleb Swanigan
df_athletics.drop([286], inplace=True) #Melo Trimble
df_athletics.drop([295], inplace=True) #Rawle Alkins
df_athletics.drop([312], inplace=True) #Hamidou Diallo
df_athletics.drop([273], inplace=True) #Malik Newman
df_athletics.drop([351], inplace=True) #Moritz Wagner
df_athletics.drop([436], inplace=True) #Bernard James
df_athletics.drop([458], inplace=True) #Jeff Taylor
df_athletics.drop([459], inplace=True) #Joe Harris
df_athletics.drop([476], inplace=True) #Ray Spalding

## Identification des doublons non repérés à cause d'une différence sur l'orthographe

In [47]:
df_athletics[df_athletics['player_name'].str.contains('[^a-zA-Z0-9^\s]', regex=True)]['player_name']

4                 J'Covan Brown
23          Darius Johnson-Odom
28       Michael Kidd-Gilchrist
43                 Kyle O'Quinn
66     Kentavious Caldwell-Pope
69      Michael Carter-Williams
83             Tim Hardaway Jr.
143                   C.J. Fair
146               P.J. Hairston
157              K.J. McDaniels
161             Johnny O'Bryant
175                 T.J. Warren
176                 C.J. Wilcox
186         Willie Cauley-Stein
201     Rondae Hollis-Jefferson
204                 R.J. Hunter
225            D'Angelo Russell
228                 J.P. Tokoto
257         Dorian Finney-Smith
302                V.J. Beachem
318                De'Aaron Fox
326        Justin Jackson (UMD)
327        Justin Jackson (UNC)
352          Derrick Walton Jr.
355         Nigel Williams-Goss
356                 D.J. Wilson
364            Keita Bates-Diop
382     Shai Gilgeous-Alexander
427                A.J. Hammons
438               C.J. McCollum
442           De'Anthony Melton
446     

In [48]:
df_athletics[df_athletics['player_name'].str.contains('Devonte', regex=True)]['player_name']

383     Devonte Graham
446    Devonte' Graham
Name: player_name, dtype: object

In [49]:
df_athletics[df_athletics['player_name'].str.contains('Dominic Pointer', regex=True)]['player_name']

481    Sir 'Dominic Pointer
Name: player_name, dtype: object

In [50]:
df_athletics[df_athletics['player_name'].str.contains('Anthony Melton', regex=True)]['player_name']

400     DeAnthony Melton
442    De'Anthony Melton
Name: player_name, dtype: object

In [51]:
df_athletics[df_athletics['player_name'].str.contains('Aaron', regex=True)]['player_name']

136       Aaron Craft
144      Aaron Gordon
197    Aaron Harrison
318      De'Aaron Fox
388     Aaron Holiday
428       Aaron White
Name: player_name, dtype: object

In [52]:
df_athletics[df_athletics['player_name'].str.contains('Hammons', regex=True)]['player_name']

427    A.J. Hammons
Name: player_name, dtype: object

In [53]:
df_athletics.loc[[383,446,400,442]]

,year,player_name,position,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,HEIGHT_WO_SHOES,HEIGHT_W_SHOES,STANDING_REACH,WEIGHT,WINGSPAN
383,2018-19,Devonte Graham,PG,4.80%,8.50,9.50,6' 0.25'',6' 1.5'',8' 0'',186.4,6' 6.25''
446,NaN,Devonte' Graham,PG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400,2018-19,DeAnthony Melton,SG-PG,5.20%,8.50,9.50,6' 2.25'',6' 3.25'',8' 3.5'',193.2,6' 8.5''
442,NaN,De'Anthony Melton,PG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Doublons à effacer : Devonte Graham, DeAnthony Melton 

In [54]:
df_athletics.drop([446,442], inplace=True)

Le seul nom dupliqué qui reste est *Tony Mitchell* qui correspond cependant à deux joueurs différents.

In [55]:
df_athletics['player_name'].duplicated().sum()

1

 # Merging de *df_uni_last_year* et de *df_athletics*

## Traitement des doublons dans df_uni_last_year

In [56]:
df_uni_last_year[df_uni_last_year['player_name'].duplicated()]

,player_name,season,school_name,conf_abbr,g,gs,mp,per,ts_pct,efg_pct,...,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,years
26,Arnett Moultrie,2011-12,Mississippi State,SEC,30,30,1073,25.2,0.610,0.561,...,15.3,23.3,3.6,1.4,5.0,0.186,4.8,0.4,5.2,1
35,Orlando Johnson,2011-12,UCSB,Big West,31,31,1067,26.0,0.560,0.527,...,12.2,30.5,4.0,1.2,5.2,0.194,6.5,-0.2,6.3,3
45,Justin Hamilton,2011-12,LSU,SEC,33,32,990,22.7,0.553,0.498,...,12.9,22.6,2.6,1.7,4.3,0.172,3.8,2.1,5.9,1
94,Romero Osby,2012-13,Oklahoma,Big 12,32,31,912,28.9,0.604,0.539,...,9.0,26.0,4.2,1.4,5.6,0.244,7.8,2.1,10.0,2
97,Colton Iverson,2012-13,Colorado State,MWC,35,35,1033,27.4,0.602,0.596,...,14.3,23.8,3.9,1.8,5.7,0.219,6.4,1.5,7.9,1
99,Arsalan Kazemi,2012-13,Oregon,Pac-12,35,27,1016,24.4,0.627,0.595,...,16.6,15.2,2.7,3.0,5.7,0.226,5.3,5.4,10.8,1
102,Alex Oriakhi,2012-13,Missouri,SEC,34,34,878,27.2,0.674,0.639,...,14.2,18.5,3.4,1.6,5.1,0.230,5.7,2.7,8.4,1
123,Rodney Hood,2013-14,Duke,ACC,35,33,1150,20.1,0.590,0.550,...,10.2,23.8,3.6,1.2,4.9,0.170,6.3,0.9,7.2,1
142,Jordan Clarkson,2013-14,Missouri,SEC,35,35,1228,20.2,0.547,0.482,...,14.3,27.9,3.2,1.2,4.4,0.142,3.8,0.5,4.3,1
151,Xavier Thames,2013-14,San Diego State,MWC,36,36,1126,26.5,0.557,0.478,...,8.2,28.6,4.7,2.8,7.5,0.267,7.2,5.2,12.4,3


In [57]:
df_uni_last_year[df_uni_last_year['player_name'].str.contains('Arnett Moultrie', regex=True)]['player_name']

25    Arnett Moultrie
26    Arnett Moultrie
Name: player_name, dtype: object

In [58]:
df_uni_last_year.loc[[25,26]]

,player_name,season,school_name,conf_abbr,g,gs,mp,per,ts_pct,efg_pct,...,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,years
25,Arnett Moultrie,2009-10,UTEP,CUSA,33,33,955,16.2,0.515,0.491,...,18.2,20.2,0.8,2.3,3.1,0.131,NaN,NaN,NaN,2
26,Arnett Moultrie,2011-12,Mississippi State,SEC,30,30,1073,25.2,0.610,0.561,...,15.3,23.3,3.6,1.4,5.0,0.186,4.8,0.4,5.2,1


In [59]:
df_uni_last_year[df_uni_last_year['player_name'].str.contains('Orlando Johnson', regex=True)]['player_name']

34    Orlando Johnson
35    Orlando Johnson
Name: player_name, dtype: object

In [60]:
df_uni_last_year.loc[[34,35]]

,player_name,season,school_name,conf_abbr,g,gs,mp,per,ts_pct,efg_pct,...,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,years
34,Orlando Johnson,2007-08,Loyola Marymount,WCC,31,30,807,NaN,0.504,0.474,...,17.4,NaN,0.8,-0.2,0.5,0.026,NaN,NaN,NaN,1
35,Orlando Johnson,2011-12,UCSB,Big West,31,31,1067,26.0,0.560,0.527,...,12.2,30.5,4.0,1.2,5.2,0.194,6.5,-0.2,6.3,3


In [61]:
df_uni_last_year[df_uni_last_year['player_name'].str.contains('Justin Hamilton', regex=True)]['player_name']

44    Justin Hamilton
45    Justin Hamilton
Name: player_name, dtype: object

In [62]:
df_uni_last_year.loc[[44,45]]

,player_name,season,school_name,conf_abbr,g,gs,mp,per,ts_pct,efg_pct,...,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,years
44,Justin Hamilton,2009-10,Iowa State,Big 12,31,31,663,18.0,0.643,0.617,...,16.4,13.6,1.6,0.9,2.5,0.150,NaN,NaN,NaN,2
45,Justin Hamilton,2011-12,LSU,SEC,33,32,990,22.7,0.553,0.498,...,12.9,22.6,2.6,1.7,4.3,0.172,3.8,2.1,5.9,1


In [63]:
df_uni_last_year.drop_duplicates(subset='player_name', keep='last', inplace=True, ignore_index=True)

In [64]:
df_uni_last_year.duplicated().sum()

0

In [65]:
df_uni_last_year[df_uni_last_year['player_name'].duplicated()]

,player_name,season,school_name,conf_abbr,g,gs,mp,per,ts_pct,efg_pct,...,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,years


## Correction des noms mal orthographiés dans *df_uni_last_year* afin de pouvoir merger *df_uni_last_year* avec *df_athletics*

In [66]:
df_uni_last_year.shape

(335, 29)

In [67]:
df_athletics.shape

(470, 11)

In [71]:
df_uni_last_year[~df_uni_last_year['player_name'].isin(df_athletics['player_name'])]

,player_name,season,school_name,conf_abbr,g,gs,mp,per,ts_pct,efg_pct,...,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,years
29,Jeffery Taylor,2011-12,Vanderbilt,SEC,36,36,1157,23.7,0.580,0.570,...,13.6,26.4,3.6,1.6,5.2,0.180,6.2,1.8,8.0,4
30,Bernard James,2011-12,Florida State,ACC,35,35,979,23.3,0.602,0.606,...,18.9,20.0,2.0,2.3,4.3,0.176,3.6,4.0,7.6,2
77,Glen Rice Jr.,2011-12,Georgia Tech,ACC,21,10,628,23.0,0.539,0.525,...,15.9,25.9,1.3,1.1,2.4,0.156,5.3,2.8,8.0,3
122,Joe Harris,2013-14,Virginia,ACC,37,37,1066,18.7,0.565,0.547,...,10.9,23.3,2.6,2.4,5.0,0.187,4.5,4.1,8.6,4
185,Sir'Dominic Pointer,2014-15,St. John's (NY),Big East,33,32,1143,24.7,0.566,0.527,...,14.2,21.0,2.8,2.4,5.2,0.180,4.6,5.5,10.1,4
240,Dennis Smith Jr.,2016-17,NC State,ACC,32,32,1114,23.1,0.563,0.520,...,17.3,27.2,3.1,0.9,4.0,0.142,5.4,1.1,6.5,1
304,Chandler Hutchison,2017-18,Boise State,MWC,31,31,961,25.9,0.575,0.528,...,16.2,33.2,2.8,2.2,5.0,0.208,5.1,3.8,8.9,4
313,Devonte' Graham,2017-18,Kansas,Big 12,39,39,1474,21.2,0.568,0.510,...,15.5,23.9,4.9,1.8,6.7,0.182,6.6,2.0,8.6,4
321,De'Anthony Melton,2016-17,USC,Pac-12,36,25,972,19.2,0.530,0.483,...,18.5,17.7,1.7,1.5,3.2,0.133,2.6,4.6,7.2,1
330,Raymond Spalding,2017-18,Louisville,ACC,36,34,995,25.6,0.568,0.550,...,13.5,22.9,2.4,2.4,4.8,0.193,4.1,4.1,8.2,3


In [74]:
df_uni_last_year[~df_uni_last_year['player_name'].isin(df_athletics['player_name'])].shape

(10, 29)

In [104]:
df_athletics[df_athletics['player_name'].str.contains('Spalding', regex=True)]['player_name']

415    Ray Spalding
Name: player_name, dtype: object

In [98]:
df_athletics.loc[481, 'player_name']

"Sir 'Dominic Pointer"

In [105]:
df_uni_last_year.loc[29, 'player_name'] = 'Jeff Taylor'
df_uni_last_year.loc[30, 'player_name'] = 'Bernard James'
df_uni_last_year.loc[77, 'player_name'] = 'Glen Rice Jr'
df_uni_last_year.loc[122, 'player_name'] = 'Joe Harris'
df_uni_last_year.loc[185, 'player_name'] = "Sir 'Dominic Pointer"
df_uni_last_year.loc[240, 'player_name'] = 'Dennis Smith Jr'
df_uni_last_year.loc[304, 'player_name'] = 'Chandler Hutchinson'
df_uni_last_year.loc[313, 'player_name'] = 'Devonte Graham'
df_uni_last_year.loc[321, 'player_name'] = 'DeAnthony Melton'
df_uni_last_year.loc[330, 'player_name'] = 'Ray Spalding'

In [106]:
df_uni_last_year[~df_uni_last_year['player_name'].isin(df_athletics['player_name'])].shape

(0, 29)

In [117]:
df_athletics_mergeable = df_athletics[df_athletics['player_name'].isin(df_uni_last_year['player_name'])]

In [119]:
df_athletics_mergeable[df_athletics_mergeable['player_name'].duplicated()]

,year,player_name,position,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,HEIGHT_WO_SHOES,HEIGHT_W_SHOES,STANDING_REACH,WEIGHT,WINGSPAN
100,2013-14,Tony Mitchell,PF-SF,5.55%,9.00,10.00,6' 7.5'',6' 8.75'',8' 10.5'',236.4,7' 2.5''


In [128]:
df_athletics_mergeable[df_athletics_mergeable['player_name'].str.contains('Tony Mitchell', regex=False)]

,year,player_name,position,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,HEIGHT_WO_SHOES,HEIGHT_W_SHOES,STANDING_REACH,WEIGHT,WINGSPAN
39,2012-13,Tony Mitchell,SG,5.70%,8.50,10.00,6' 5.25'',6' 6.25'',8' 4'',216.4,6' 10''


## Merging de *df_uni_last_year* & *df_athletics_mergeable*

In [364]:
data_X = df_uni_last_year.set_index('player_name').join(df_athletics_mergeable.set_index('player_name'))

In [365]:
data_X.head()

,season,school_name,conf_abbr,g,gs,mp,per,ts_pct,efg_pct,fg3a_per_fga_pct,...,year,position,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,HEIGHT_WO_SHOES,HEIGHT_W_SHOES,STANDING_REACH,WEIGHT,WINGSPAN
player_name,,,,,,,,,,,,,,,,,,,,,
Anthony Davis,2011-12,Kentucky,SEC,40,40,1281,35.1,0.654,0.628,0.059,...,2012-13,PF-C,7.90%,9.00,8.50,6' 9.25'',6' 10.5'',9' 0'',221.8,7' 5.5''
Michael Kidd-Gilchrist,2011-12,Kentucky,SEC,40,39,1245,21.2,0.570,0.511,0.156,...,2012-13,SF,7.00%,9.00,10.25,6' 5.75'',6' 7.5'',8' 8.5'',232.8,7' 0''
Bradley Beal,2011-12,Florida,SEC,37,37,1267,22.0,0.575,0.525,0.473,...,2012-13,SG,6.00%,8.50,9.00,6' 3.25'',6' 4.75'',8' 4'',201.8,6' 8''
Dion Waiters,2011-12,Syracuse,Big East,37,0,891,26.3,0.565,0.534,0.317,...,2012-13,SG,8.50%,8.50,9.50,6' 2.5'',6' 4'',8' 2'',221.0,6' 7.25''
Thomas Robinson,2011-12,Kansas,Big 12,39,39,1242,27.4,0.549,0.512,0.027,...,2012-13,PF,5.00%,9.75,10.50,6' 7.75'',6' 8.75'',8' 10'',244.2,7' 3.25''


# Traitement sur data_X

## Lower le nom des colonnes

In [366]:
data_X.columns = [x.lower() for x in data_X.columns]

## Sélection d'une seule position

In [367]:
data_X['position'] = data_X['position'].apply(lambda x:x.split('-')[0])

## Conversion des grandeurs en cm

In [368]:
def data_to_convert(x):
    try:
        return convert_height_2(*list(map(lambda x:x.strip(), str(x).split("'")))[:2])
    except:
        return 0 #np.nan

def convert_height(feet,inches):
    h_ft = float(feet)
    h_inch = float(inches)
    h_inch += h_ft * 12
    h_cm = round(h_inch * 2.54, 1)
    return h_cm

def convert_height_2(*args):
    if len(args)>1:
        h_ft = float(args[0])
        h_inch = float(args[1])
    else:
        h_ft = 0
        h_inch = float(args[0])
        
    h_inch += h_ft * 12
    h_cm = round(h_inch * 2.54, 1)
    return int(h_cm*10)

In [369]:
data_X_test = data_X.copy()

In [370]:
data_X_test.columns

Index(['season', 'school_name', 'conf_abbr', 'g', 'gs', 'mp', 'per', 'ts_pct',
       'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct',
       'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct',
       'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm',
       'years', 'year', 'position', 'body_fat_pct', 'hand_length',
       'hand_width', 'height_wo_shoes', 'height_w_shoes', 'standing_reach',
       'weight', 'wingspan'],
      dtype='object')

In [374]:
data_X['hand_length'] = data_X['hand_length'].apply(data_to_convert)
data_X['hand_width'] = data_X['hand_width'].apply(data_to_convert)
data_X['height_wo_shoes'] = data_X['height_wo_shoes'].apply(data_to_convert)
data_X['height_w_shoes'] = data_X['height_w_shoes'].apply(data_to_convert)
data_X['standing_reach'] = data_X['standing_reach'].apply(data_to_convert)
data_X['wingspan'] = data_X['wingspan'].apply(data_to_convert)
data_X

,season,school_name,conf_abbr,g,gs,mp,per,ts_pct,efg_pct,fg3a_per_fga_pct,...,year,position,body_fat_pct,hand_length,hand_width,height_wo_shoes,height_w_shoes,standing_reach,weight,wingspan
player_name,,,,,,,,,,,,,,,,,,,,,
Anthony Davis,2011-12,Kentucky,SEC,40,40,1281,35.1,0.654,0.628,0.059,...,2012-13,PF,7.90%,229,216,2064,2096,2743,221.8,2273
Michael Kidd-Gilchrist,2011-12,Kentucky,SEC,40,39,1245,21.2,0.570,0.511,0.156,...,2012-13,SF,7.00%,229,260,1975,2019,2654,232.8,2134
Bradley Beal,2011-12,Florida,SEC,37,37,1267,22.0,0.575,0.525,0.473,...,2012-13,SG,6.00%,216,229,1911,1949,2540,201.8,2032
Dion Waiters,2011-12,Syracuse,Big East,37,0,891,26.3,0.565,0.534,0.317,...,2012-13,SG,8.50%,216,241,1892,1930,2489,221.0,2013
Thomas Robinson,2011-12,Kansas,Big 12,39,39,1242,27.4,0.549,0.512,0.027,...,2012-13,PF,5.00%,248,267,2026,2051,2692,244.2,2216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ray Spalding,2017-18,Louisville,ACC,36,34,995,25.6,0.568,0.550,0.056,...,2018-19,PF,5.55%,235,229,2057,2089,2769,215.4,2254
Kevin Hervey,2017-18,Texas-Arlington,Sun Belt,33,25,1064,23.7,0.553,0.517,0.417,...,2018-19,PF,5.15%,235,260,1994,2026,2718,211.6,2222
Thomas Welsh,2017-18,UCLA,Pac-12,33,33,1096,20.3,0.577,0.552,0.335,...,2017-18,PF,10.90%,229,267,2121,2146,2781,254.0,2134


## Retire le % de *body_fat_pct*

In [379]:
data_X['body_fat_pct'] = data_X['body_fat_pct'].apply(lambda x:str(x).replace('%', '')) 

In [409]:
data_X.loc['Kris Dunn','body_fat_pct'] = 'nan'
data_X.loc['Markelle Fultz','body_fat_pct'] = 'nan'

In [412]:
#[^a-zA-Z0-9^\s]
data_X['body_fat_pct'].replace('nan', np.nan, inplace=True)

## Optimisation du type des données

In [414]:
data_X.dtypes

season               object
school_name          object
conf_abbr            object
g                     int64
gs                    int64
mp                    int64
per                 float64
ts_pct              float64
efg_pct             float64
fg3a_per_fga_pct    float64
fta_per_fga_pct     float64
pprod               float64
orb_pct             float64
drb_pct             float64
trb_pct             float64
ast_pct             float64
stl_pct             float64
blk_pct             float64
tov_pct             float64
usg_pct             float64
ows                 float64
dws                 float64
ws                  float64
ws_per_40           float64
obpm                float64
dbpm                float64
bpm                 float64
years                 int64
year                 object
position             object
body_fat_pct         object
hand_length           int64
hand_width            int64
height_wo_shoes       int64
height_w_shoes        int64
standing_reach      

In [424]:
data_X['weight'] = data_X['weight'].apply(lambda x:str(x))

In [426]:
data_X[data_X['weight'].str.contains('-')]['weight']

player_name
Alex Len          -
Markelle Fultz    -
Name: weight, dtype: object

In [430]:
data_X.loc['Alex Len','weight'] = 0
data_X.loc['Markelle Fultz','weight'] = 0

In [431]:
data_X['weight'].astype('float64')

player_name
Anthony Davis             221.8
Michael Kidd-Gilchrist    232.8
Bradley Beal              201.8
Dion Waiters              221.0
Thomas Robinson           244.2
                          ...  
Ray Spalding              215.4
Kevin Hervey              211.6
Thomas Welsh              254.0
George King               218.6
Kostas Antetokounmpo      194.8
Name: weight, Length: 335, dtype: float64

In [433]:
data_X['body_fat_pct'].astype('float64')

player_name
Anthony Davis              7.90
Michael Kidd-Gilchrist     7.00
Bradley Beal               6.00
Dion Waiters               8.50
Thomas Robinson            5.00
                          ...  
Ray Spalding               5.55
Kevin Hervey               5.15
Thomas Welsh              10.90
George King                7.25
Kostas Antetokounmpo       5.00
Name: body_fat_pct, Length: 335, dtype: float64

In [434]:
data_X['hand_length'].astype('int32')
data_X['hand_width'].astype('int32')
data_X['height_wo_shoes'].astype('int32')
data_X['height_w_shoes'].astype('int32')
data_X['standing_reach'].astype('int32')
data_X['wingspan'].astype('int32')

player_name
Anthony Davis             2273
Michael Kidd-Gilchrist    2134
Bradley Beal              2032
Dion Waiters              2013
Thomas Robinson           2216
                          ... 
Ray Spalding              2254
Kevin Hervey              2222
Thomas Welsh              2134
George King               2121
Kostas Antetokounmpo      2191
Name: wingspan, Length: 335, dtype: int32

In [435]:
data_X.to_csv('../nba_forecast/data/dataset_X_cleaned.csv')